In [1]:
#Librairies 
import pandas as pd
import requests
import time


In [ ]:
# Importation de jeux_confirmes_final
jeux_confirmes_final = pd.read_csv(r"data/raw/jeux_confirmes_final.csv")
jeux_confirmes_final.head()

,AppID,Name,Release date,Estimated owners,Price,Peak CCU,User score,Positive,Negative,Metacritic score,...,Developers,Publishers,Genres,Tags,Categories,Windows,Mac,Linux,Website,published
0,20200,Galactic Bowling,2008-10-21,0 - 20000,19.99,0,0,6,11,0,...,Perpetual FX Creative,Perpetual FX Creative,"Casual,Indie,Sports","Indie,Casual,Sports,Bowling","Single-player,Multi-player,Steam Achievements,...",1,0,0,http://www.galacticbowling.net,1
1,655370,Train Bandit,2017-10-12,0 - 20000,0.99,0,0,53,5,0,...,Rusty Moyher,Wild Rooster,"Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...","Single-player,Steam Achievements,Full controll...",1,1,0,http://trainbandit.com,1
2,1732930,Jolt Project,2021-11-17,0 - 20000,4.99,0,0,0,0,0,...,Campião Games,Campião Games,"Action,Adventure,Indie,Strategy","Aventure, Robots, Militaire, Stratégie, Tanks,...",Single-player,1,0,0,NaN,1
3,1355720,Henosis™,2020-07-23,0 - 20000,5.99,0,0,3,0,0,...,Odd Critter Games,Odd Critter Games,"Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...","Single-player,Full controller support",1,1,1,https://henosisgame.com/,1
4,1139950,Two Weeks in Painland,2020-02-03,0 - 20000,0.00,0,0,50,8,0,...,Unusual Games,Unusual Games,"Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...","Single-player,Steam Achievements",1,1,0,https://www.unusual-games.com/home/,1


In [ ]:

def get_steam_price(appid):
    """Récupère le prix actuel d'un jeu Steam via l'API officielle."""
    
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=fr"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"❌ Erreur HTTP {response.status_code} pour AppID {appid}")
            return None
        
        data = response.json()
        
        # Vérifier si l'AppID existe
        if str(appid) not in data or not data[str(appid)]["success"]:
            print(f"⚠ AppID {appid} introuvable ou indisponible")
            return None
        
        game_data = data[str(appid)]["data"]
        
        # Vérifier si le jeu est gratuit
        if game_data.get("is_free", False):
            print(f"✔ AppID {appid} → Gratuit")
            return 0.0
        
        # Récupérer le prix
        price_overview = game_data.get("price_overview")
        if price_overview:
            # Le prix est en centimes, on divise par 100
            price = price_overview["final"] / 100
            currency = price_overview.get("currency", "EUR")
            print(f"✔ AppID {appid} → {price} {currency}")
            return price
        else:
            print(f"⚠ AppID {appid} → Pas de prix disponible")
            return None
    
    except Exception as e:
        print(f"🔥 Erreur pour AppID {appid}: {e}")
        return None


In [13]:
def update_steam_prices(delay=1.5):
    """Met à jour la colonne 'Price' directement dans jeux_confirmes_final."""
    
    global jeux_confirmes_final
    
    print(f"🔄 Mise à jour des prix pour {len(jeux_confirmes_final)} jeux...\n")
    
    for idx in jeux_confirmes_final.index:
        appid = jeux_confirmes_final.at[idx, 'AppID']
        print(f"[{idx+1}/{len(jeux_confirmes_final)}] ", end="")
        
        new_price = get_steam_price(appid)
        
        # Mettre à jour directement dans le DataFrame
        if new_price is not None:
            jeux_confirmes_final.at[idx, 'Price'] = new_price
        
        # Délai pour éviter de surcharger l'API Steam
        time.sleep(delay)
    
    # Statistiques
    nb_updated = jeux_confirmes_final['Price'].notna().sum()
    nb_free = (jeux_confirmes_final['Price'] == 0.0).sum()
    nb_failed = jeux_confirmes_final['Price'].isna().sum()
    
    print(f"\n✅ Mise à jour terminée!")
    print(f"   - Prix mis à jour: {nb_updated}")
    print(f"   - Jeux gratuits: {nb_free}")
    print(f"   - Échecs: {nb_failed}")

In [16]:
jeux_confirmes_final.head()

,AppID,Name,Release date,Estimated owners,Price,Peak CCU,User score,Positive,Negative,Metacritic score,...,Developers,Publishers,Genres,Tags,Categories,Windows,Mac,Linux,Website,published
0,20200,Galactic Bowling,2008-10-21,0 - 20000,19.99,0,0,6,11,0,...,Perpetual FX Creative,Perpetual FX Creative,"Casual,Indie,Sports","Indie,Casual,Sports,Bowling","Single-player,Multi-player,Steam Achievements,...",1,0,0,http://www.galacticbowling.net,1
1,655370,Train Bandit,2017-10-12,0 - 20000,0.99,0,0,53,5,0,...,Rusty Moyher,Wild Rooster,"Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...","Single-player,Steam Achievements,Full controll...",1,1,0,http://trainbandit.com,1
2,1732930,Jolt Project,2021-11-17,0 - 20000,3.99,0,0,0,0,0,...,Campião Games,Campião Games,"Action,Adventure,Indie,Strategy","Aventure, Robots, Militaire, Stratégie, Tanks,...",Single-player,1,0,0,NaN,1
3,1355720,Henosis™,2020-07-23,0 - 20000,0.00,0,0,3,0,0,...,Odd Critter Games,Odd Critter Games,"Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...","Single-player,Full controller support",1,1,1,https://henosisgame.com/,1
4,1139950,Two Weeks in Painland,2020-02-03,0 - 20000,0.00,0,0,50,8,0,...,Unusual Games,Unusual Games,"Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...","Single-player,Steam Achievements",1,1,0,https://www.unusual-games.com/home/,1
